In [1]:
pip install chromedriver-autoinstaller


C:\Users\User\Web_Crawling\nerdy>doskey defects4j=perl C:\Users\User\defects4j\framework\bin\defects4j $* 
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install selenium


C:\Users\User\Web_Crawling\nerdy>doskey defects4j=perl C:\Users\User\defects4j\framework\bin\defects4j $* 
Note: you may need to restart the kernel to use updated packages.


In [3]:
import chromedriver_autoinstaller
from selenium import webdriver
import time

class Driver:
    @staticmethod
    def driver_init():
        chromedriver_autoinstaller.install()
        options = webdriver.ChromeOptions()
#         options.add_argument('headless') # jupyter notebook 환경에서 주석처리
        options.add_argument('lang=ko_KR')
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36")
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-notifications')
        options.add_argument('--disable-popup-blocking')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-browser-side-navigation')
        options.add_argument('disable-gpu')

        driver = webdriver.Chrome(options=options)
        driver.set_window_size(1600, 1100)

        return driver

In [4]:
driver = Driver.driver_init()

In [5]:
def find_product_paths(urls):
    product_paths = []
    
    ## 상의/자켓/하의 분류 루프
    for (url, pages_cnt) in urls:
        ## 각 옷 분류의 페이지 갯수 루프
        for page_num in range(pages_cnt):
            try:
                driver.get(url + "&page=" + str(page_num+1))
            except:
                continue

            products = driver.find_elements('xpath', '//ul[@class="prdNeo prd_set"]/li')
            products_cnt = len(products)

            ## 각 product의 path를 product_paths 리스트에 저장
            for i in range(products_cnt):
                product = driver.find_elements('xpath', '//ul[@class="prdNeo prd_set"]/li')[i]
                product_path = product.find_element('xpath', './div/div[@class="thumbnail on"]/a').get_attribute('href')
                product_paths.append(product_path)
                
    return product_paths

In [ ]:
def crawl_pages(product_paths):
    
    base_dict= {"baseCat":[], "baseName": [], "baseCode": [], "baseUrl": [], "colorName": [],
            "baseGender": [], "price": [], "basePrice": [], "colorFabric": [], "imageUrl": [],
            "size": [], "isAvailable": []
            }
    
    ## 드라이버 초기화
    driver = Driver.driver_init()
    ## 각 product에 대한 정보 추출
    for product_path in product_paths:
        driver.get(product_path)

        cat = 'top'
        if "category/24" in product_path or "category/52" in product_path:
            cat = 'top'
        elif "category/26" in product_path:
            cat = 'bottom'

        baseURL = product_path
        base_gender = 0
        text_info = driver.find_element('xpath', '//div[@class="infoArea"]')
        imageURL = driver.find_element('xpath','//img[@class="BigImage "]').get_attribute('src')

        base_name = text_info.find_element('xpath', './div[@class="headingArea"]/h2').text
        base_code = ""
        color_name = ""
        ## color가 명시되지 않은 옷은 제외(패키지 상품)
        if "_" in base_name:
            base_code = base_name.split("_")[0]
            ## base_code 앞에 협업 브랜드 텍스트 제거
            if "] " in base_code:
                base_code = base_code.split("] ")[1]

            ## color_name 뒤에 협업 브랜드 텍스트 제거
            color_name = base_name.split("_")[1]
            if "[" in color_name:
                color_name = color_name.split("[")[0]
        else:
            continue

        base_price = ""
        price = ""
        color_fabric = ""
        trs = text_info.find_elements("xpath",'./div[2]/table/tbody/tr') #body의 table 항목 갯수                       
        if len(trs) == 5:
            base_price = text_info.find_element("xpath",'.//strong[@id="span_product_price_text"]').text
            price = text_info.find_element("xpath",'.//span[@id="span_product_price_sale"]').text
            color_fabric = text_info.find_element("xpath",'./div[2]/table/tbody/tr[5]/td/span').text
        elif len(trs) == 4:
            base_price = text_info.find_element("xpath",'.//strong[@id="span_product_price_text"]').text
            price = base_price
            color_fabric = text_info.find_element("xpath",'./div[2]/table/tbody/tr[4]/td/span').text
        elif len(trs) == 3:
            base_price = text_info.find_element("xpath",'.//strong[@id="span_product_price_text"]').text
            price = base_price
            color_fabric = ""
        elif len(trs) == 2:
            base_price = ""
            price = ""
            color_fabric = ""
            

        size_elements = text_info.find_elements("xpath",'.//optgroup[@label="사이즈"]/option')
        size = "M"
        is_available = True

        for element in size_elements:
            size = element.text.replace(" [품절]","")
            if "[품절]" in element.text:
                is_available = False
            else:
                is_available = True

            base_dict["baseCat"].append(cat)
            base_dict["baseName"].append(base_name)
            base_dict["baseCode"].append(base_code)
            base_dict["baseUrl"].append(baseURL)
            base_dict["colorName"].append(color_name)
            base_dict["baseGender"].append(0)
            base_dict["price"].append(price)
            base_dict["basePrice"].append(base_price)
            base_dict["colorFabric"].append(color_fabric)
            base_dict["imageUrl"].append(imageURL)
            base_dict["size"].append(size)
            base_dict["isAvailable"].append(is_available)
            
    return base_dict

In [ ]:
import pandas as pd

In [6]:
urls = [
        ('https://whoisnerdy.com/product/list.html?cate_no=30', 15), #상의
        ('https://whoisnerdy.com/product/list.html?cate_no=61', 5), #자켓
        ('https://whoisnerdy.com/product/list.html?cate_no=31', 7), #하의
         ]

product_paths = find_product_paths(urls)
print(len(product_paths))

778
